In [23]:
# Импортируем необходимые библиотеки
import pandas as pd
from datetime import date, time

In [56]:
# Настроим отображение строк и колонок
pd.set_option('display.max_rows', 12)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

In [6]:
#Читаем исходный датасет
dst = pd.read_csv('query_result_2021-11-12T18_36_18.815853Z.csv')

In [19]:
#Создаем датафрейм
dst = pd.DataFrame(dst)

In [20]:
# Смотрим, что получилось - ничего неожиданного. Те же колонки, что и были экспортированы из базы.
dst

,flight_id,flight_no,scheduled_departure,arrival_airport,aircraft_code,flight_amount,sold_seats,total_ac_capacity,f_duration,fuel_rate
0,136119,PG0252,2017-01-03T10:05:00Z,SVO,733,1653000.0,113.0,130,100.0,2.6
1,136120,PG0252,2017-02-24T10:05:00Z,SVO,733,1605400.0,109.0,130,100.0,2.6
2,136122,PG0252,2017-01-08T10:05:00Z,SVO,733,1431000.0,97.0,130,100.0,2.6
3,136130,PG0252,2017-02-13T10:05:00Z,SVO,733,1556600.0,107.0,130,100.0,2.6
4,136131,PG0252,2017-01-28T10:05:00Z,SVO,733,1812800.0,124.0,130,100.0,2.6
...,...,...,...,...,...,...,...,...,...,...
122,136936,PG0480,2017-01-02T09:25:00Z,EGO,SU9,632400.0,82.0,97,50.0,1.7
123,136937,PG0480,2017-01-03T09:25:00Z,EGO,SU9,626100.0,81.0,97,50.0,1.7
124,136951,PG0480,2017-02-02T09:25:00Z,EGO,SU9,720600.0,90.0,97,50.0,1.7
125,136953,PG0480,2017-02-27T09:25:00Z,EGO,SU9,765300.0,97.0,97,50.0,1.7


In [9]:
# посмотрим какие типы воздушных судов присутствуют в датасете
dst.aircraft_code.unique()
# Итого: Boeing 737-300, SSJ-100. 

array(['733', 'SU9'], dtype=object)

Итак, мы получили из базы определенный набор данных, но его недостаточно, чтобы решить поставленную задачу.
Напомним себе, что задача - определить прибыльность или убыточность авиарейсов и на этом основании отобрать "кандидатов на вылет".
Давайте определимся, что такое прибыльность или убыточность рейса. 
- #### Прибыль - это разница между абсолютной прибылью от проданных билетов на рейс и суммой затрат на выполнение рейса. 
Если разница положительная - рейс прибыльный, иначе убыточный.
Но как нам оценить совокупность затрат на выполнение рейса? Есть методики рассчета себестоимости авиарейсов, но объективно мы не сможем ими воспользоваться в полной мере, т.к. у нас нет доступа к внутренней финансовой информации авиакомпании(зарплаты экипажей, ежемесячные лизинговые платежи за воздушное судно). Тем не менее, мы знаем ряд параметров, которые мы можем оценить с определенной точностью: 
- средний расход топлива в зависимости от типа ВС (рассчитывается в тоннах на час работы двигателей. Даннный параметр более предпочтителен, т.к. учитывает не только расстояние между точками А и Б, но и еще маневры на взлет/посадку, по итогу наземное расстояние получается больше); 
- стоимость аэропортового обслуживания(сюда входят сбор за взлет/посадку, сбор за авиабезопасность(обеспечение диспетчерского обслуживания в зоне действия аэропорта в воздухе и на земле), сбор за посадку/высадку пассажиров, сборы за обслуживание самолета и экипажа и т.д.);
- затраты на обслуживание пассажиров(354 руб./чел входит в перечень услуг аэропортового обслуживания, но зависит от количества пассажиров, поэтому вынесено в отдельную статью затрат);
- процент заполненности судна (данный параметр позволяет оценить разницу между проданными билетами на рейс и вместительностью судна в процентах, введен для удобства).

Стоимость затрат на рейс рассчитана по статьям в таблице "Расчет_расходов_на_полет.xlsx", которая загружена на Github. Рассчеты произведены на основании официального прайс-листа взятого с сайта международного аэропорта Анапы: aaq.aero. Получившиеся суммы использованы для рассчетов по всем рейсам, т.к. в соответствии с географией полетов стоимость аналогичных услуг в аэропорту города Белгород, примерно на 40% выше (это обусловлено меньшим количеством обслуживаемых рейсов), при этом стоимость по прайсу Шереметьево на 40% ниже. Таким образом получаем, в среднем, указанный прайс.
Все вышеописанное есть переменные статьи расходов, т.к. они зависят от заполняемости воздушного судна, длительности рейса(взлетной и посадочной массы) и прочих изменяющихся значений.
Помимо этого есть, так называемые, постоянные расходы:
 - зарплата экипажа;
 - затраты на владение ВС(лизинговые платежи);
 - затраты на поддержание летной годности(будущий ремонт).
 
 Данная статья расходов расчитывается в руб./час и умножается на длительность рейса. К сожалению, у нас нет доступа к данного рода информации, поэтому я рассчитал приблизительную стоимость по каждому разделу на основании информации из открытых источников.
 Для рассчета затрат на топливо я взял среднюю стоимость авиационного керосина за три зимних месяца 2017г. (январ, февраль, декабрь) и умножил на средний часовой расход ВС. Стоимость топлива в Анапе за указанный период взята с сайта Федерального агентства воздушного транспорта: https://favt.gov.ru/stat-date-gsm-price/?id=7329 
 
 В процессе подготовки расчетных данных я обнаружил интересный факт: затраты на поддержание летной годности SSJ100 больше чем в 2.5 раза выше чем у Boeing или Airbus. При том, что мест в салоне у них больше. Как в последствии выяснилось, это обусловлено плохой логистикой запчастей и высокой стоимостью содержания самолета. По стоимости расчета рейса есть хорошая статья https://osdy.ru/blog/fly-costs/. Я выборочно проверил приведенные цифры, они практически полностью сошлись с указанными в статье рассчетами.
 
 В рассчетах я не стал учитывать прочие расходы на поддержание сотрудников офиса, налоговые сборы(принимаем, что прибыль уже указана после налогов), затраты на страховку, затраты на систему бронирования билетов и т.д. 
 В целом, полученная финансовая модель достаточно наглядно показывает прибыльность рейсов, несмотря на отсутствие части расходов. По полученным данным можно делать определенные выводы. 

In [78]:
#введем дополнительные колонки, необходимые для решения задачи
dst['fuel_rate'] = dst.aircraft_code.apply(lambda x: 2.6 if x == '733' else 1.7) #расход топлива в тоннах в час
dst['fixed_costs'] = dst.aircraft_code.apply(lambda x: 130844 if x == '733' else 184270) #постоянные расходы (рассчет приведен в табл)
dst['pass_mntns'] = dst.sold_seats.apply(lambda x: x * 354) # расходы на обслуживание пассажиров в аэропорту
dst['airport_taxes'] = dst.aircraft_code.apply(lambda x: 456212 if x == '733' else 418608) #аэропортовые сборы
dst['total_flight_amount'] = dst.flight_amount - round((((((dst.fuel_rate*42697) \
     + dst.fixed_costs)/60)*(dst.f_duration)) + dst.pass_mntns + dst.airport_taxes), 1) # итоговая прибыль от продаж билетов за вычетом всех расходов
dst.scheduled_departure = pd.to_datetime(dst.scheduled_departure) # приводим время вылета к формату datetime
dst['capacity_percentage'] = round((dst.sold_seats*100) / dst.total_ac_capacity, 1) # рассчитываем процент заполняемости воздушного судна 

In [79]:
dst.sort_values(by='total_flight_amount', ascending=True)

,flight_id,flight_no,scheduled_departure,arrival_airport,aircraft_code,flight_amount,sold_seats,total_ac_capacity,f_duration,fuel_rate,total_flight_amount,week_day,capacity_percentage,fixed_costs,pass_mntns,airport_taxes
102,136807,PG0480,2017-02-23 09:25:00+00:00,EGO,SU9,531000.0,68.0,97,50.0,1.7,-125725.8,3,70.1,184270,24072.0,418608
77,136642,PG0480,2017-01-30 09:25:00+00:00,EGO,SU9,531000.0,64.0,97,50.0,1.7,-124309.8,0,66.0,184270,22656.0,418608
109,136844,PG0480,2017-02-28 09:25:00+00:00,EGO,SU9,575100.0,79.0,97,50.0,1.7,-85519.8,1,81.4,184270,27966.0,418608
116,136887,PG0480,2017-01-20 09:25:00+00:00,EGO,SU9,595200.0,78.0,97,50.0,1.7,-65065.8,4,80.4,184270,27612.0,418608
120,136922,PG0480,2017-02-11 09:25:00+00:00,EGO,SU9,607800.0,76.0,97,50.0,1.7,-51757.8,5,78.4,184270,26904.0,418608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,136540,PG0194,2017-01-17 06:10:00+00:00,NOZ,733,NaN,NaN,130,305.0,2.6,NaN,1,NaN,130844,NaN,456212
64,136544,PG0194,2017-01-31 06:10:00+00:00,NOZ,733,NaN,NaN,130,305.0,2.6,NaN,1,NaN,130844,NaN,456212
65,136546,PG0194,2017-01-03 06:10:00+00:00,NOZ,733,NaN,NaN,130,305.0,2.6,NaN,1,NaN,130844,NaN,456212
66,136560,PG0194,2017-01-24 06:10:00+00:00,NOZ,733,NaN,NaN,130,305.0,2.6,NaN,1,NaN,130844,NaN,456212


In [58]:
# Посмотрим номера всех рейсов в датасете
dst.flight_no.unique()

array(['PG0252', 'PG0194', 'PG0480'], dtype=object)

Особого интереса заслуживает рейс PG0194 в Новокузнецк(NOZ).
Данный рейс выполнялся по понедельникам при этом самолет прилетал в Анапу накануне утром и оставался там до момента возврата на следующий день. При этом, по данному рейсу нет никакой информации по количеству проданных билетов и, как следствие, прибыльности. Данный факт наводит на мысль, что продажи билетов осуществлялись сторонним сервисом и в базу не попали. Учитывая данное заключение, данный рейс имеет смысл оставить, но загрузить недостающие значения в базу и после этого проводить анализ.

In [83]:
# Количество выполненных рейсов в Новокузнецк
dst[dst.flight_no == 'PG0194'].flight_id.nunique()
# Рейсы выполнялись по понедельникам. Количество рейсов укладывается в два зимних месяца: январь и февраль 2017г.

9

In [89]:
# Параметр week_day обозначает день недели от 0(воскресенье) до 6(суббота)
# Посчитав количество выполненных рейсов PG0480 по дням недели, мы видим, что рейсы в Белгород выполнялись почти каждый день.
dst[dst.flight_no == 'PG0480'].week_day.value_counts()

1    9
6    9
0    9
2    8
4    8
5    8
3    8
Name: week_day, dtype: int64

In [93]:
# Посчитав количество выполненных рейсов PG0252 по дням недели, мы видим, что рейсы в г.Москва, 
# а/п Шереметьево выполнялись с такой же частотой как и PG0480. В таком же количестве.
dst[dst.flight_no == 'PG0252'].week_day.value_counts()

1    9
6    9
0    9
4    8
5    8
2    8
3    8
Name: week_day, dtype: int64

In [101]:
# Приведем основные цифры и проанализируем полученные результаты
print('Средняя заполняемость рейса PG0480: ', round(dst[dst.flight_no == 'PG0480'].capacity_percentage.mean(), 1),'%')
print('Средняя заполняемость рейса PG0252: ', round(dst[dst.flight_no == 'PG0252'].capacity_percentage.mean(), 1),'%')
print('Средняя прибыль рейса PG0480: ', round(dst[dst.flight_no == 'PG0480'].total_flight_amount.mean(), 1))
print('Средняя прибыль рейса PG0252: ', round(dst[dst.flight_no == 'PG0252'].total_flight_amount.mean(), 1))
print('Количество проданных билетов на рейс PG0480: ', round(dst[dst.flight_no == 'PG0480'].sold_seats.sum(), 1))
print('Количество проданных билетов на рейс PG0252: ', round(dst[dst.flight_no == 'PG0252'].sold_seats.sum(), 1))
print('Сумма прибыли от всех проданных билетов на рейс PG0252: ', round(dst[dst.flight_no == 'PG0480'].total_flight_amount.sum(), 1))
print('Сумма прибыли от всех проданных билетов на рейс PG0252: ', round(dst[dst.flight_no == 'PG0252'].total_flight_amount.sum(), 1))

Средняя заполняемость рейса PG0480:  93.0 %
Средняя заполняемость рейса PG0252:  87.0 %
Средняя прибыль рейса PG0480:  45855.8
Средняя прибыль рейса PG0252:  743226.6
Количество проданных билетов на рейс PG0480:  5321.0
Количество проданных билетов на рейс PG0252:  6674.0
Сумма прибыли от всех проданных билетов на рейс PG0252:  2705491.8
Сумма прибыли от всех проданных билетов на рейс PG0252:  43850367.7


Давайте сравним цифры, полученные в ходе анализа датасета:

- Средняя заполняемость рейса PG0480:  93.0 %
- Средняя заполняемость рейса PG0252:  87.0 %

При средней заполняемости рейса PG0480 ,ольшей, чем у рейса PG0252, количество проданных билетов у PG0252 больше в виду большей вместимости судна. Получается 90 билетов PG0480, против 113 на PG0252.
Обратимся к средней прибыльности рейса за вычетом основных расходов:

- Средняя прибыль рейса PG0480:  45855.8
- Средняя прибыль рейса PG0252:  743226.6

При этом, средняя чистая прибыль на рейс до Москвы более чем в 16 раз выше, чем в Белгород.
Посчитаем количество всех проданных билетов за исследуемый период.
- Количество проданных билетов на рейс PG0480:  5321.0
- Количество проданных билетов на рейс PG0252:  6674.0

Разница составляет всего 20% и она обусловлена вместимостью воздушных судов на обоих направлениях.
А теперь посчитаем сумму чистой прибыли от продажи всех билетов за рассматриваемый период по обоим направлениям:

- Сумма прибыли от всех проданных билетов на рейс PG0252:  2705491.8
- Сумма прибыли от всех проданных билетов на рейс PG0252:  43850367.7

Не трудно догадаться, что прибыль отличается более чем в 16 раз. И это при том, что у меня не все расходы учтены. В реальности, рейсы в Белгород будут абсолютно убыточными.

На основании приведенных выше рассчетов, можно заключить, что с точки зрения оптимизации расходов авиакомпании, стоит отказаться от рейса PG0480.

При этом стоит заметить, что если сократить количество рейсов в Белгород до 3-х - 4-х в неделю и заменив воздушное судно на Boeing 737-300 можно сделать данное направление прибыльным за счет уменьшения затрат на содержание и обслуживание ВС, при высокой степени заполняемости.